# Working with CDP Transcripts

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database and file store usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Find all transcripts

Simple query the transcript table!

In [2]:
transcripts = pd.DataFrame(db.select_rows_as_list("transcript"))
transcripts

,confidence,created,event_id,file_id,id
0,0.881332,2019-04-22 06:56:27.283578,3807e904-a7f6-44a9-8116-667aac02ec93,db6e8e0e-338e-4ff3-b7ac-15bc9aa6d7ec,3563074c-c791-4e2c-83e1-2172977be487
1,0.947337,2019-04-21 23:31:38.809855,226d8033-666c-49aa-831d-37d04d693106,43b2d231-5a0e-4c5b-876e-51859e86f0da,658bfe6b-6efc-4efc-b7c9-de53a1a98651
2,0.924026,2019-04-21 23:30:06.600107,bcdff355-e045-45ee-b1f5-477cb518a27e,0aceb6c8-3f7c-494f-9a97-cf5c319acb77,a5ae7d7d-3bc2-4c3a-b07a-7829a8abf1c9
3,0.930153,2019-04-21 23:58:05.245933,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,ebbd9727-d3ef-41ea-82b5-1cda7d1ca050,bb31c1eb-021d-4eb4-8c34-ec97e8871828
4,0.933456,2019-04-21 23:24:47.975906,614c9534-810f-48b7-b375-afc6e14024cd,480cb0a9-0c5f-4791-8795-ca71583eb785,d147a3ac-2b08-462f-b2ba-c7c18c182c2d
5,0.927340,2019-04-21 23:23:31.532067,1ffb5920-3c23-4084-b287-cef74c9c56c8,0099dfe6-ee18-4a16-a9b5-e7a053d9582d,d4e6bb99-6624-44af-a25f-dfde9f69bf29


### Join file, event, and body information

While the above results are somewhat useful, we should probably merge information from the other tables too...

In [3]:
# Get the other tables
events = pd.DataFrame(db.select_rows_as_list("event"))
bodies = pd.DataFrame(db.select_rows_as_list("body"))
files = pd.DataFrame(db.select_rows_as_list("file"))

# Merge the tables
events = events.merge(bodies, left_on="body_id", right_on="id", suffixes=("_event", "_body"))
transcripts = transcripts.merge(files, left_on="file_id", right_on="id", suffixes=("_transcript", "_file"))
transcripts = transcripts.merge(events, left_on="event_id", right_on="id_event", suffixes=("_transcript", "_event"))
transcripts

,confidence,created_transcript,event_id,file_id,id_transcript,created_file,filename,id_file,uri,body_id,created_event,event_datetime,id_event,source_uri,video_uri,created_body,description,id_body,name
0,0.881332,2019-04-22 06:56:27.283578,3807e904-a7f6-44a9-8116-667aac02ec93,db6e8e0e-338e-4ff3-b7ac-15bc9aa6d7ec,3563074c-c791-4e2c-83e1-2172977be487,2019-04-22 06:56:25.586170,eb448680c9c0cd7132d91b70a36d1a1c9f6bf7bc5eb148...,db6e8e0e-338e-4ff3-b7ac-15bc9aa6d7ec,gs://stg-cdp-seattle.appspot.com/eb448680c9c0c...,8309112f-85c6-458a-8ef8-879907068177,2019-04-22 06:56:26.878303,2016-12-13T00:00:00,3807e904-a7f6-44a9-8116-667aac02ec93,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...,2019-04-22 06:56:26.298234,None,8309112f-85c6-458a-8ef8-879907068177,"Civil Rights, Utilities, Economic Development ..."
1,0.947337,2019-04-21 23:31:38.809855,226d8033-666c-49aa-831d-37d04d693106,43b2d231-5a0e-4c5b-876e-51859e86f0da,658bfe6b-6efc-4efc-b7c9-de53a1a98651,2019-04-21 23:31:36.233402,be4a521638e7f738bb2f8441ebac43aa2c4ae8d461511d...,43b2d231-5a0e-4c5b-876e-51859e86f0da,gs://stg-cdp-seattle.appspot.com/be4a521638e7f...,c28e1141-60f2-421d-9c17-e629b57e8890,2019-04-21 23:31:38.209946,2019-04-11T00:00:00,226d8033-666c-49aa-831d-37d04d693106,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/li...,2019-04-21 23:31:37.572810,None,c28e1141-60f2-421d-9c17-e629b57e8890,Select Committee on the Library Levy
2,0.924026,2019-04-21 23:30:06.600107,bcdff355-e045-45ee-b1f5-477cb518a27e,0aceb6c8-3f7c-494f-9a97-cf5c319acb77,a5ae7d7d-3bc2-4c3a-b07a-7829a8abf1c9,2019-04-21 23:30:05.227034,c9a9a1cd9e9f30e894ccba46ed40b4285ace66196acb19...,0aceb6c8-3f7c-494f-9a97-cf5c319acb77,gs://stg-cdp-seattle.appspot.com/c9a9a1cd9e9f3...,f993dbf3-47d3-4632-85ea-424852247a4b,2019-04-21 23:30:06.231758,2017-06-09T00:00:00,bcdff355-e045-45ee-b1f5-477cb518a27e,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/su...,2019-04-21 23:30:05.890924,None,f993dbf3-47d3-4632-85ea-424852247a4b,Sustainability and Transportation Committee
3,0.930153,2019-04-21 23:58:05.245933,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,ebbd9727-d3ef-41ea-82b5-1cda7d1ca050,bb31c1eb-021d-4eb4-8c34-ec97e8871828,2019-04-21 23:58:03.624545,52d797171b74b68246b4ad0f2c4131c125c3a9338688ea...,ebbd9727-d3ef-41ea-82b5-1cda7d1ca050,gs://stg-cdp-seattle.appspot.com/52d797171b74b...,6f38a688-2e96-4e33-841c-883738f9f03d,2019-04-21 23:58:04.832481,2017-06-27T00:00:00,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ge...,2019-04-21 23:58:04.378827,None,6f38a688-2e96-4e33-841c-883738f9f03d,"Gender Equity, Safe Communities & New Americans"
4,0.933456,2019-04-21 23:24:47.975906,614c9534-810f-48b7-b375-afc6e14024cd,480cb0a9-0c5f-4791-8795-ca71583eb785,d147a3ac-2b08-462f-b2ba-c7c18c182c2d,2019-04-21 23:24:47.128465,b7c5868853cb63b212100f9e0f2c19f88389e0175c1714...,480cb0a9-0c5f-4791-8795-ca71583eb785,gs://stg-cdp-seattle.appspot.com/b7c5868853cb6...,318d0a2a-93d1-417b-aa26-e37ad61b81e8,2019-04-21 23:24:47.698886,2015-06-24T00:00:00,614c9534-810f-48b7-b375-afc6e14024cd,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-04-21 23:24:47.435263,None,318d0a2a-93d1-417b-aa26-e37ad61b81e8,Finance and Culture Committee
5,0.927340,2019-04-21 23:23:31.532067,1ffb5920-3c23-4084-b287-cef74c9c56c8,0099dfe6-ee18-4a16-a9b5-e7a053d9582d,d4e6bb99-6624-44af-a25f-dfde9f69bf29,2019-04-21 23:23:29.888746,165f1f333b3607748d8beca97e59a6b273fde4e2cf7b57...,0099dfe6-ee18-4a16-a9b5-e7a053d9582d,gs://stg-cdp-seattle.appspot.com/165f1f333b360...,44a794de-6e1d-43dd-ac9f-317924345bdb,2019-04-21 23:23:30.958242,2017-12-06T00:00:00,1ffb5920-3c23-4084-b287-cef74c9c56c8,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ed...,2019-04-21 23:23:30.575342,None,44a79

### Downloading and reading a transcript

A manifest is great, but you probably want to work with the transcripts locally.

In [4]:
# Download to local
save_path = fs.download_file(transcripts.loc[0]["filename"])
save_path

[INFO: file_store:  54 2019-04-24 18:53:13,230] Stored external resource copy: /Users/jacksonb/Desktop/active/cdp/cdptools/examples/eb448680c9c0cd7132d91b70a36d1a1c9f6bf7bc5eb14811cc5abc68f78d2894_transcript_0.txt


PosixPath('/Users/jacksonb/Desktop/active/cdp/cdptools/examples/eb448680c9c0cd7132d91b70a36d1a1c9f6bf7bc5eb14811cc5abc68f78d2894_transcript_0.txt')

In [5]:
# Read the transcript
with open(save_path, "r") as read_in:
    print(read_in.read()[:101])

Greetings, welcome to the Civil Rights utilities economic development in arts committee. This is our 


### Handling multiple transcripts for a single event

A single event may have multiple versions of a transcripts. In this case you can decide how you want to filter them down, but generally, a decent solution is to group by `id_event`  then choose the most recent. You can do this with `pandas.Series.idxmax()`.

[stackoverflow](https://stackoverflow.com/questions/10202570/pandas-dataframe-find-row-where-values-for-column-is-maximal)

In [6]:
# Group rows
most_recent_transcripts = []
grouped = transcripts.groupby("event_id")
for name, group in grouped:
    most_recent = group.loc[group["created_transcript"].idxmax()]
    most_recent_transcripts.append(most_recent)
    
most_recent = pd.DataFrame(most_recent_transcripts)
most_recent

,confidence,created_transcript,event_id,file_id,id_transcript,created_file,filename,id_file,uri,body_id,created_event,event_datetime,id_event,source_uri,video_uri,created_body,description,id_body,name
3,0.930153,2019-04-21 23:58:05.245933,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,ebbd9727-d3ef-41ea-82b5-1cda7d1ca050,bb31c1eb-021d-4eb4-8c34-ec97e8871828,2019-04-21 23:58:03.624545,52d797171b74b68246b4ad0f2c4131c125c3a9338688ea...,ebbd9727-d3ef-41ea-82b5-1cda7d1ca050,gs://stg-cdp-seattle.appspot.com/52d797171b74b...,6f38a688-2e96-4e33-841c-883738f9f03d,2019-04-21 23:58:04.832481,2017-06-27T00:00:00,0e3bd59c-3f07-452c-83cf-e9eebeb73af2,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ge...,2019-04-21 23:58:04.378827,None,6f38a688-2e96-4e33-841c-883738f9f03d,"Gender Equity, Safe Communities & New Americans"
5,0.927340,2019-04-21 23:23:31.532067,1ffb5920-3c23-4084-b287-cef74c9c56c8,0099dfe6-ee18-4a16-a9b5-e7a053d9582d,d4e6bb99-6624-44af-a25f-dfde9f69bf29,2019-04-21 23:23:29.888746,165f1f333b3607748d8beca97e59a6b273fde4e2cf7b57...,0099dfe6-ee18-4a16-a9b5-e7a053d9582d,gs://stg-cdp-seattle.appspot.com/165f1f333b360...,44a794de-6e1d-43dd-ac9f-317924345bdb,2019-04-21 23:23:30.958242,2017-12-06T00:00:00,1ffb5920-3c23-4084-b287-cef74c9c56c8,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ed...,2019-04-21 23:23:30.575342,None,44a794de-6e1d-43dd-ac9f-317924345bdb,"Education, Equity, and Governance Committee"
1,0.947337,2019-04-21 23:31:38.809855,226d8033-666c-49aa-831d-37d04d693106,43b2d231-5a0e-4c5b-876e-51859e86f0da,658bfe6b-6efc-4efc-b7c9-de53a1a98651,2019-04-21 23:31:36.233402,be4a521638e7f738bb2f8441ebac43aa2c4ae8d461511d...,43b2d231-5a0e-4c5b-876e-51859e86f0da,gs://stg-cdp-seattle.appspot.com/be4a521638e7f...,c28e1141-60f2-421d-9c17-e629b57e8890,2019-04-21 23:31:38.209946,2019-04-11T00:00:00,226d8033-666c-49aa-831d-37d04d693106,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/li...,2019-04-21 23:31:37.572810,None,c28e1141-60f2-421d-9c17-e629b57e8890,Select Committee on the Library Levy
0,0.881332,2019-04-22 06:56:27.283578,3807e904-a7f6-44a9-8116-667aac02ec93,db6e8e0e-338e-4ff3-b7ac-15bc9aa6d7ec,3563074c-c791-4e2c-83e1-2172977be487,2019-04-22 06:56:25.586170,eb448680c9c0cd7132d91b70a36d1a1c9f6bf7bc5eb148...,db6e8e0e-338e-4ff3-b7ac-15bc9aa6d7ec,gs://stg-cdp-seattle.appspot.com/eb448680c9c0c...,8309112f-85c6-458a-8ef8-879907068177,2019-04-22 06:56:26.878303,2016-12-13T00:00:00,3807e904-a7f6-44a9-8116-667aac02ec93,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...,2019-04-22 06:56:26.298234,None,8309112f-85c6-458a-8ef8-879907068177,"Civil Rights, Utilities, Economic Development ..."
4,0.933456,2019-04-21 23:24:47.975906,614c9534-810f-48b7-b375-afc6e14024cd,480cb0a9-0c5f-4791-8795-ca71583eb785,d147a3ac-2b08-462f-b2ba-c7c18c182c2d,2019-04-21 23:24:47.128465,b7c5868853cb63b212100f9e0f2c19f88389e0175c1714...,480cb0a9-0c5f-4791-8795-ca71583eb785,gs://stg-cdp-seattle.appspot.com/b7c5868853cb6...,318d0a2a-93d1-417b-aa26-e37ad61b81e8,2019-04-21 23:24:47.698886,2015-06-24T00:00:00,614c9534-810f-48b7-b375-afc6e14024cd,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-04-21 23:24:47.435263,None,318d0a2a-93d1-417b-aa26-e37ad61b81e8,Finance and Culture Committee
2,0.924026,2019-04-21 23:30:06.600107,bcdff355-e045-45ee-b1f5-477cb518a27e,0aceb6c8-3f7c-494f-9a97-cf5c319acb77,a5ae7d7d-3bc2-4c3a-b07a-7829a8abf1c9,2019-04-21 23:30:05.227034,c9a9a1cd9e9f30e894ccba46ed40b4285ace66196acb19...,0aceb6c8-3f7c-494f-9a97-cf5c319acb77,gs://stg-cdp-seattle.appspot.com/c9a9a1cd9e9f3...,f993dbf3-47d3-4632-85ea-424852247a4b,2019-04-21 23:30:06.231758,2017-06-09T00:00:00,bcdff355-e045-45ee-b1f5-477cb518a27e,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/su...,2019-04-21 23:30:05.890924,None,f99

### A function to download the most recent transcripts

To make gathering the most recent transcripts easier, let's wrap this example up with a function to do all this for us.

**Note:** The function defined below is available in the `cdptools.utils` module.

`from cdptools.utils import download_most_recent_transcripts`

In [7]:
from pathlib import Path
from typing import Optional, Union

from cdptools.databases import Database
from cdptools.file_stores import FileStore


def download_most_recent_transcripts(db: Database, fs: FileStore, save_dir: Optional[Union[str, Path]] = None) -> Path:
    """
    Download the most recent versions of event transcripts.

    :param db: An already initialized `Database` object to query against.
    :param fs: An alreay initialized `FileStore` object to query against.
    :param save_dir: Path to a directory to save the transcripts and the dataset manifest.
    :return: Fully resolved path where transcripts and dataset manifest were stored.
    """
    # Use current directory is None provided
    if save_dir is None:
        save_dir = "."
    
    # Resolve save directory
    save_dir = Path(save_dir).resolve()
    
    # Make the save directory if not already exists
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Get transcript dataset
    transcripts = pd.DataFrame(db.select_rows_as_list("transcript"))
    events = pd.DataFrame(db.select_rows_as_list("event"))
    bodies = pd.DataFrame(db.select_rows_as_list("body"))
    files = pd.DataFrame(db.select_rows_as_list("file"))
    events = events.merge(bodies, left_on="body_id", right_on="id", suffixes=("_event", "_body"))
    transcripts = transcripts.merge(files, left_on="file_id", right_on="id", suffixes=("_transcript", "_file"))
    transcripts = transcripts.merge(events, left_on="event_id", right_on="id_event", suffixes=("_transcript", "_event"))
    
    # Group
    most_recent_transcripts = []
    grouped = transcripts.groupby("event_id")
    for name, group in grouped:
        most_recent = group.loc[group["created_transcript"].idxmax()]
        most_recent_transcripts.append(most_recent)
    
    most_recent = pd.DataFrame(most_recent_transcripts)
    
    # Begin storage
    most_recent.apply(
        lambda r: fs.download_file(r["filename"], save_dir / r["filename"], overwrite=True), 
        axis=1
    )
    
    # Write manifest
    most_recent.to_csv(save_dir / "transcript_manifest.csv", index=False)
    
    return save_dir

In [8]:
save_dir = download_most_recent_transcripts(db, fs, "most_recent_seattle_transcripts")
save_dir

[INFO: file_store:  54 2019-04-24 18:53:15,382] Stored external resource copy: /Users/jacksonb/Desktop/active/cdp/cdptools/examples/most_recent_seattle_transcripts/52d797171b74b68246b4ad0f2c4131c125c3a9338688eaf83109ae719fff2bee_transcript_0.txt
[INFO: file_store:  54 2019-04-24 18:53:15,543] Stored external resource copy: /Users/jacksonb/Desktop/active/cdp/cdptools/examples/most_recent_seattle_transcripts/165f1f333b3607748d8beca97e59a6b273fde4e2cf7b577f0f1b2fab631952a2_transcript_0.txt
[INFO: file_store:  54 2019-04-24 18:53:15,710] Stored external resource copy: /Users/jacksonb/Desktop/active/cdp/cdptools/examples/most_recent_seattle_transcripts/be4a521638e7f738bb2f8441ebac43aa2c4ae8d461511d95571c4b49f5ad299c_transcript_0.txt
[INFO: file_store:  54 2019-04-24 18:53:15,876] Stored external resource copy: /Users/jacksonb/Desktop/active/cdp/cdptools/examples/most_recent_seattle_transcripts/eb448680c9c0cd7132d91b70a36d1a1c9f6bf7bc5eb14811cc5abc68f78d2894_transcript_0.txt
[INFO: file_stor

PosixPath('/Users/jacksonb/Desktop/active/cdp/cdptools/examples/most_recent_seattle_transcripts')